# Aprendizado Supervisionado e K Vizinhos mais Próximos

## Introdução

Nós utilizaremos a base de dados de taxa de rejeição da indústria de telecomunicações para essa lista de exercícios. A base de dados tem o nome `Orange_Telecom_Churn_Data.csv`. Nesse notebook carregaremos a base de dados, faremos algum pré-processamento e usaremos a técnica k-NN para predizer a taxa de rejeição baseado nas características mensuradas.

## Exercício 1

* Comece importando os dados através do pandas. Examine as colunas e os dados
* Note que os dados contêm estado, código de área e telefone. Você acha que esses atributos são interessantes para construir nosso modelo de classificação? Por que?

Não utilizaremos esses atributos para essa base, então eles podem ser removidos do dataframe (método `drop`).

In [1]:
import pandas as pd

data = pd.read_csv('data/Orange_Telecom_Churn_Data.csv').drop(['state', 'area_code', 'phone_number'], axis=1)

## Exercício 2

* Note que algumas das colunas são categóricas e algumas são *float*. Esses atributos precisam ser numéricos para usar os algoritmos que aprenderemos no curso.
* Finalmente, o algoritmo K-Vizinhos mais próximos necessita de dados escalonados. Escalone os dados utilizando um dos métodos aprendidos em aula.

In [2]:
import warnings
warnings.filterwarnings('ignore', module='sklearn')

from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

#binarizer
bin_cols = ['intl_plan', 'voice_mail_plan', 'churned']
binarizers = []
for s in bin_cols:
    binarizers.append(LabelBinarizer())
    data[s] = binarizers[-1].fit_transform(data[s])

#scale
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns.values)

## Exercício 3

* Separe as colunas de atributos (todas menos `churned`) da  coluna de rótulo (`churned`). Isso criará duas tabelas.
* Aplique o método `.fit()` do K-nearest neighbors com um valor de `n_neighbors=3` para essa base de dados e verifique o resultado com o método `.predict()` na mesma base.

In [3]:
x_cols = [x for x in data.columns if x != 'churned']

X_data = data_scaled[x_cols].values
y_data = data_scaled['churned'].values

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=1111)

In [4]:
from sklearn.neighbors import KNeighborsClassifier


model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_data, y_data)

model.score(X_data, y_data)

0.94220000000000004

## Exercício 4

Ainda não ensinamos formas de medir o erro, mas a acurácia é um conceito simples de entender--é a porcentagem de rótulos que foram corretamente classificados:

$acc = \frac{corretos}{total}$

* Escreva uma função para calcular a acurácia usando os rótulos verdadeiros e os preditos.
* Usando a função, calcule a acurácia do K-nn nos dados.

In [5]:
def accuracy(real, predict):
    a = (real == predict).sum()
    b    = real.shape[0] 
    return (a/b)


acc = accuracy(y_data, model.predict(X_data))
acc

0.94220000000000004

## Exercício 5

* Repita a aplicação do modelo K-nearest neighbors mas com o parâmetro `weights=distance`. Calcule a acurácia da função criada por você.
* Repita a aplicação do modelo com `n_neighbors=3`, `weights=uniform` e `p=1` para utilizar a distância de Manhattan. Verifique os resultados.

Quando as distâncias ponderadas são utilizadas para a parte 1 dessa questão, vocês obterão acurácia de 1.0. Por que isso ocorre? *Dica:* o KNN usa diretamente os dados para fazer a predição.

In [ ]:
model2 = KNeighborsClassifier(n_neighbors=3, weights='distance')
model2.fit(X_data, y_data)

model3 = KNeighborsClassifier(n_neighbors=3, weights='uniform', p=1)
model3.fit(X_data, y_data)

#model2.score(X_data, y_data), model3.score(X_data, y_data)
accuracy(y_data, model2.predict(X_data)), accuracy(y_data, model3.predict(X_data))

(1.0, 0.9456)

## Exercício 6

* Teste o modelo KNN utilizando valores de `n_neighbors` na faixa de 1 a 20. Deixe o restante dos parâmetros como o padrão. Armazene os valores de vizinhos e acurácia em uma lista no formato [(k, acuracia)].
* Plote um gráfico do tipo *scatter* da acurácia vs vizinhos. O que acontece com `n_neighbors=1`? Por que isso ocorre?

In [ ]:
score_list = list()

'''escreva no bloco abaixo'''
for k in range(1,20+1):
    m = KNeighborsClassifier(n_neighbors=k)
    m.fit(X_data, y_data)
    
    score_list.append([k, accuracy(y_data, m.predict(X_data))])
'''fim do código'''

score_df = pd.DataFrame(score_list, columns=['k', 'accuracy'])

In [ ]:
# Import libraries to make the plot

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')
sns.set_palette('dark')

ax = score_df.set_index('k').plot()

ax.set(xlabel='k', ylabel='accuracy')
ax.set_xticks(range(1, 21));

plt.scatter(score_df['k'],score_df['accuracy'])